In [1]:
import torchaudio
import numpy as np
import scipy.signal
from transform import standard_scale

# Cargar el audio
audio, sample_rate = torchaudio.load('data/mobile/test_clicker.mp4')
audio, sample_rate

(tensor([[ 2.5042e-08,  9.0870e-08,  1.8204e-07,  ..., -2.7731e-02,
          -2.7269e-02, -2.7010e-02]]),
 44100)

In [2]:
# Normalizar el audio
scaled_audio = standard_scale(audio[0])  # Asumiendo un canal mono
scaled_audio.mean(), scaled_audio.std()

(tensor(4.5920e-09), tensor(1.))

In [3]:
# Aplicar la transformada de Fourier
transformed = np.fft.fft(scaled_audio)
transformed

array([5.18192271e-03  +0.j        , 1.01710533e+02+181.06093701j,
       2.77745305e+02 -68.49700401j, ..., 1.75391948e+02 +99.05084083j,
       2.77745305e+02 +68.49700401j, 1.01710533e+02-181.06093701j])

In [4]:
# Calcular la magnitud del espectro
magnitude = np.abs(transformed)
magnitude

array([5.18192271e-03, 2.07673049e+02, 2.86066940e+02, ...,
       2.01428411e+02, 2.86066940e+02, 2.07673049e+02])

In [5]:
# Convertir índices a frecuencias
frequencies = np.fft.fftfreq(len(scaled_audio), 1 / sample_rate)
frequencies

array([ 0.        ,  0.03879856,  0.07759713, ..., -0.11639569,
       -0.07759713, -0.03879856])

In [6]:
# Suavizar el espectro (opcional)
window = scipy.signal.windows.hamming(51)
magnitude = scipy.signal.convolve(magnitude, window, mode='same')
magnitude

array([3360.41733176, 3618.09102207, 3877.39077158, ..., 4137.9649276 ,
       3877.38566454, 3618.08585894])

In [7]:
# Establecer un umbral basado en un múltiplo del valor medio o mediano
threshold = np.median(magnitude) * 2  # Ajusta el factor según sea necesario
threshold

56.26296172314879

In [8]:
# Detectar picos
peaks, _ = scipy.signal.find_peaks(magnitude, height=threshold)
peaks

array([     43,      48,     143, ..., 1136497, 1136592, 1136597])

In [9]:
# Encontrar las frecuencias de los picos
peak_frequencies = frequencies[peaks]
peak_frequencies

array([ 1.66833826,  1.86233108,  5.54819468, ..., -5.54819468,
       -1.86233108, -1.66833826])

In [10]:
# Identificar el pico más prominente o el que coincide con la frecuencia esperada del clicker
clicker_peak_freq = peak_frequencies[np.argmax(magnitude[peaks])]

print(f"La frecuencia del clicker se estima que está alrededor de: {clicker_peak_freq} Hz")

La frecuencia del clicker se estima que está alrededor de: -62.038904138513516 Hz


In [11]:
from scipy.signal import butter, filtfilt, find_peaks

# Diseñar un filtro de paso de banda para la frecuencia del clicker
def design_bandpass_filter(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Aplicar el filtro de paso de banda a la señal de audio
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = design_bandpass_filter(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
# Definir el rango de la banda alrededor de la frecuencia del clicker
tolerance = 5  # Esto es solo un ejemplo, ajusta según la anchura de la banda deseada
filtered_audio = bandpass_filter(scaled_audio, clicker_peak_freq - tolerance, clicker_peak_freq + tolerance, sample_rate, order=2)

# Detectar eventos de clicker en el tiempo
magnitude_filtered = np.abs(filtered_audio)
threshold_clicker_time = np.median(magnitude_filtered) * 1.5  # Ajusta este valor según sea necesario
clicker_events, _ = find_peaks(magnitude_filtered, height=threshold_clicker_time)

# Mapear los índices de eventos a tiempos reales
clicker_times = clicker_events / sample_rate

# Ahora tienes los tiempos en los que ocurren los eventos del clicker
print(clicker_times)


In [11]:




# Identificar el pico de frecuencia del clicker
# Supongamos que ya conocemos que el clicker tiene una frecuencia aproximada
clicker_freq = 2000  # Frecuencia aproximada del clicker en Hz
tolerance = 50  # Tolerancia en Hz alrededor de la frecuencia del clicker

# Encontrar el índice del pico en el rango de frecuencias
indices = (frequencies > (clicker_freq - tolerance)) & (frequencies < (clicker_freq + tolerance))
peak_index = np.argmax(magnitude[indices])

# Frecuencia real del pico
real_peak_freq = frequencies[indices][peak_index]

print(f"El pico del clicker se encuentra en: {real_peak_freq} Hz")


El pico del clicker se encuentra en: 2035.7606630067569 Hz


In [ ]:

# Asumiendo que 'magnitude' y 'frequencies' se obtuvieron como se describió anteriormente

# Suavizar el espectro (opcional)
# window = scipy.signal.windows.hamming(51)
# smoothed_magnitude = scipy.signal.convolve(magnitude, window, mode='same')

# Establecer un umbral basado en un múltiplo del valor medio o mediano
threshold = np.median(magnitude) * 5  # Ajusta el factor según sea necesario

# Detectar picos
peaks, _ = scipy.signal.find_peaks(magnitude, height=threshold)

# Encontrar las frecuencias de los picos
peak_frequencies = frequencies[peaks]

# Identificar el pico más prominente o el que coincide con la frecuencia esperada del clicker
clicker_peak_freq = peak_frequencies[np.argmax(magnitude[peaks])]

print(f"La frecuencia del clicker se estima que está alrededor de: {clicker_peak_freq} Hz")
